In [1]:
import sklearn as sk
sk.__version__

'0.19.1'

In [2]:
%%script false
!pip install scikit-learn --upgrade
!pip install -U scikit-learn

In [3]:
import numpy as np
import pandas as pd
from sklearn.pipeline import make_pipeline, make_union
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor
from sklearn.base import BaseEstimator, TransformerMixin

In [50]:
# Load data
filename="./data/train.csv"
data = pd.read_csv(filename, sep="\t", index_col='id')
print(len(data))
print(data.columns)
#msg = "Reading the data ({} rows). Columns: {}"
data.describe()
data.info()
#print(msg.format(len(data), data.columns))


852885
Index(['question_id', 'subreddit', 'question_utc', 'question_text',
       'question_score', 'answer_utc', 'answer_text', 'answer_score'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 852885 entries, 330435 to 2646893
Data columns (total 8 columns):
question_id       852885 non-null object
subreddit         852885 non-null object
question_utc      852885 non-null int64
question_text     852885 non-null object
question_score    852885 non-null int64
answer_utc        852885 non-null int64
answer_text       852885 non-null object
answer_score      852885 non-null int64
dtypes: int64(4), object(4)
memory usage: 58.6+ MB


In [49]:
data.head()

,question_id,subreddit,question_utc,question_text,question_score,answer_utc,answer_text,answer_score
id,,,,,,,,
330435,f48a5420fa6a66ecec95365dd67bdc99,AskReddit,1512086400,All my buddies love Pacific Rim and always tal...,179,1512086616,"I didn't really care for it, I thought the pac...",70
1944975,f48a5420fa6a66ecec95365dd67bdc99,AskReddit,1512086400,All my buddies love Pacific Rim and always tal...,179,1512090694,"I thought it was hilarious, and I didn't reali...",24
2218735,f48a5420fa6a66ecec95365dd67bdc99,AskReddit,1512086400,All my buddies love Pacific Rim and always tal...,179,1512090763,"&gt; Pacific Rim The porno, or the studio rel...",11
2133251,f48a5420fa6a66ecec95365dd67bdc99,AskReddit,1512086400,All my buddies love Pacific Rim and always tal...,179,1512091349,I enjoyed it more than most movies of that gen...,42
2133252,f48a5420fa6a66ecec95365dd67bdc99,AskReddit,1512086400,All my buddies love Pacific Rim and always tal...,179,1512133012,"Its terrible, its clearly made to be a self aw...",1


In [35]:
# Select the columns (feel free to select more)
#X = data.loc[:, ['question_text', 'answer_text']]
y = data.loc[:, "answer_score"]
X = data
X.columns
X = X.drop('answer_score',axis='columns')

In [37]:
%%script false
def load_data(filename):
    data = pd.read_csv(filename, sep="\t", index_col='id')
    msg = "Reading the data ({} rows). Columns: {}"
    print(msg.format(len(data), data.columns))
    # Select the columns (feel free to select more)
    X = data.loc[:, ['question_text', 'answer_text']]
    try:
        y = data.loc[:, "answer_score"]
    except KeyError: # There are no answers in the test file
        return X, None
    return X, y
#X, y = load_data(filename="../input/kaggledays-warsaw/train.csv")
X, y = load_data(filename="./data/train.csv")

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [39]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(639663, 7) (639663,)
(213222, 7) (213222,)


In [40]:
def rmsle(y, y0):
    assert len(y) == len(y0)
    return np.sqrt(
        np.mean((np.log1p(y) - np.log1p(y0)) ** 2)
    )

In [41]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns, orient=None):
        super(FeatureSelector, self).__init__()
        self.columns = columns

    def fit(self, X, *args, **kwargs):
        return self

    def transform(self, data, *args, **kwargs):
        return data[self.columns].values

def build_model():
    process_data = make_union(
        make_pipeline(
            FeatureSelector("question_text"),
            TfidfVectorizer(max_features=10, token_pattern="\w+"),
        ),
        make_pipeline(
            FeatureSelector("answer_text"),
            TfidfVectorizer(max_features=10, token_pattern="\w+"),
        ),
    )

    model = make_pipeline(
         process_data,
         SGDRegressor(),
    )
    return model

In [46]:
print(type(X_train))

<class 'pandas.core.frame.DataFrame'>


In [42]:
%%time
# FIT THE MODEL
model = build_model()
model.fit(X_train, np.log1p(y_train))

/home/batman/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


CPU times: user 30 s, sys: 232 ms, total: 30.2 s
Wall time: 29.4 s


In [43]:
%%time
y_train = model.predict(X_train)
y_test = model.predict(X_test)

CPU times: user 34.6 s, sys: 44 ms, total: 34.6 s
Wall time: 33.6 s


In [44]:
y_train_theor = np.expm1(y_train)
y_test_theor = np.expm1(y_test)
print()
print("Training set")
print("RMSLE:   ", rmsle(y_train, y_train_theor))

print("Test set")
print("RMSLE:   ", rmsle(y_test, y_test_theor))


Training set
RMSLE:    0.5511275577920076
Test set
RMSLE:    0.551289433584618


In [45]:
%%time

#X_val, _ = load_data('../input/kaggledays-warsaw/test.csv')
X_val, _ = load_data('./data/test.csv')

solution = pd.DataFrame(index=X_val.index)
solution['answer_score'] = np.expm1(model.predict(X_val))
solution.to_csv('submission_baseline.csv')

Reading the data (663082 rows). Columns: Index(['question_id', 'subreddit', 'question_utc', 'question_text',
       'question_score', 'answer_utc', 'answer_text'],
      dtype='object')
CPU times: user 32.4 s, sys: 200 ms, total: 32.6 s
Wall time: 30.4 s
